In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [2]:
eps, t, r, th, ph = var('eps','t','r','th','ph')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega')(r,th)

nu = function('nu')(r)
lamb = function('lamb')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne')(r,eps)

In [3]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i][j][k] = 0.5*sum(geuu[i,dd]*(derivative(gdd[dd,j],coords[k])
                                                  + derivative(gdd[dd,k],coords[j])
                                                  - derivative(gdd[j,k],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i][j][k][l] = (derivative(chris[i][j][l],coords[k])
                                      - derivative(chris[i][j][k],coords[l])
                                      + sum(chris[i][k][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][k][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd][i][dd][k] for dd in range(0,4))
            
    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    eins = matrix(SR, 4, 4)
    eins = ricci - 0.5*gedd*s_curv
    
    return {'inverse_metric': simplify(geuu),
            'christoffel': simplify(chris),
            'riemann': simplify(riem),
            'ricci': simplify(ricci),
            'curvature': simplify(s_curv),
            'einstein': simplify(eins)}

In [4]:
start_time = time.time()

gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = matrix(SR, 4, 4)
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

tensors = cosmotensors(gedd)
geuu = tensors['inverse_metric']
ricci = tensors['ricci']
s_curv = tensors['curvature']
eins = tensors['einstein']                                                                               #G_\mu\nu e

deins = matrix(SR, 4, 4)
ddeins = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        deins[i,j] = diff(eins[i,j],eps)
        ddeins[i,j] = diff(eins[i,j],eps,eps)
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.214718103408813 seconds ---


In [5]:
print("ge0 = ")
print(gdd)
print("K1 = ")
print(K1)
print("K2 = ")
print(K2)
print("ge = ")
print(gedd)

ge0 = 
[     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]
K1 = 
[                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]
K2 = 
[2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r

In [6]:
# Para imprimir "más bonito":

show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]

'K2 = ' [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

'ge = ' [(r^2*omega(r, th)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*eps^2 - e^nu(r)                                                                   0                                                                   0                                      eps*r^2*omega(r, th)*sin(th)^2]
[                                                                  0                              2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                   0                                                                   0]
[                                                                  0                                                                   0                                          2*eps^2*r^2*k(r, th) + r^2                                                                   0]
[                                     eps*r^2*omega(r, th)*sin(th)^2                                                                   0                                                                   0                      2*eps^2*r^2*k(r, th)*sin(th)^2 + r^2*sin(th)^2]

In [7]:
#4-velocities of the fluid for any epsilon

ue = [Ne, 0, 0, Ne*Omegae]
u = [exp(-nu/2), 0, 0, 0]

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                            #Eq 4.20

In [8]:
show("Ne0 = ", Nevalue)
show("Omegae0 = ", Omegaevalue)

'Ne0 = ' e^(-1/2*nu(r))

'Omegae0 = ' 0

In [10]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10')(r)
u20 = function('u20')(r)

Omega1 = function('Omega1')(eps)
Omega2 = function('Omega2')(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,4):
    u1[i] = diff(ue[i],eps).subs(sustituciones)
    u2[i] = diff(ue[i],eps,eps).subs(sustituciones)

Ue = [0, 0, 0, 0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[i]*gedd[i,j]

for i in range(0,4):
    geUeue = Ue[i]*ue[i]

dgeUeue = diff(geUeue, eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
print(u10value)
for i in range(0,4):
    u1[i].subs(u10==u10value)                                                                              #Eq 4.21
print(u1)
#ddgeUeue = diff(geUeue.expr(), eps, eps).subs(sustituciones) == 0
#u20value = solve(ddgeUeue, u20)[0].rhs()
#s8 = u20 == u20value
#u2.apply_map(lambda cmp: cmp.subs(u20==u20value).subs(u10==u10value))                                      #Eq 4.22

r1
[u10(r), 0, 0, Omega1(eps)*e^(-1/2*nu(r))]


In [57]:
show(u1.display())
show(u2.display())

AttributeError: 'list' object has no attribute 'display'

In [ ]:
deins.apply_map(lambda cmp: cmp.subs(sustituciones))                                             #d/de(G_\mu\nu)|e=0
ddeins.apply_map(lambda cmp: cmp.subs(sustituciones))                                        #d^2/de^2(G_\mu\nu)|e=0

In [ ]:
Ee = function('Ee')(r,eps)
Pe = function('Pe')(r,eps)

eins0 = M.tensor_field(0,2, 'eins0')
Tedd = M.tensor_field(0,2, 'Tedd')
Tdd = M.tensor_field(0,2, 'Tdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        eins0[i,j] = eins[i,j]
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j]

eins0.apply_map(lambda cmp: cmp.subs(sustituciones))                                                       #G_\mu\nu
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                                         #T_\mu\nu

In [ ]:
show("Te0 =", Tdd[:])

In [ ]:
kappa = 8*pi
E = function('E')(r)
P = function('P')(r)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))
E_tt = solve(eins0[0,0].expr()==kappa*Tdd[0,0].expr(),E)[0].rhs()                                            #Eq 3.7
P_rr = solve(eins0[1,1].expr()==kappa*Tdd[1,1].expr(),P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2].expr()==kappa*Tdd[2,2].expr(),P)[0].rhs()
P_phph = solve(eins0[3,3].expr()==kappa*Tdd[3,3].expr(),P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [ ]:
show("E = ", E_tt)
show("Prr = ", P_rr)
show("ddnu =", ddnu)

In [ ]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [ ]:
show(LatexExpr("dlamb = "), dlamb)
show(LatexExpr("dnu = "), dnu)

In [ ]:
h0 = function('h0')(r,th)
k0 = function('k0')(r,th)
m0 = function('m0')(r,th)

h2 = function('h2')(r,th)
k2 = function('k2')(r,th)
m2 = function('m2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [ ]:
E1 = function('E1')(r)
E2 = function('E2')(r)
P1 = function('P1')(r)
P2 = function('P2')(r)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

s15 = m == m0 + m2*(3*cos(th)^2-1)/2
s16 = k == k0 + k2*(3*cos(th)^2-1)/2
s17 = h == h0 + h2*(3*cos(th)^2-1)/2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17)

In [ ]:
dTdd = M.tensor_field(0,2,'dTdd')
ddTdd = M.tensor_field(0,2,'ddTdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        dTdd[i,j] = diff(Tedd[i,j].expr(),eps)
        ddTdd[i,j] = diff(Tedd[i,j].expr(),eps,eps)
dTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                              #d/de(T_\mu\nu)|e=0
ddTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                         #d^2/de^2(T_\mu\nu)|e=0

In [ ]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1]," = ", kappa*dTdd[1,1])
show(deins[2,2]," = ", kappa*dTdd[2,2])
show(deins[3,3]," = ", kappa*dTdd[3,3])

P1 = solve(deins[1,1].expr() == kappa*dTdd[1,1].expr(),P1)[0].rhs()

show("De aquí sacamos P1 -> P1(r) = ", P1)

In [ ]:
show(deins[0,0]," = ", kappa*dTdd[0,0])

E1 = solve(deins[0,0].expr() == kappa*dTdd[0,0].expr(),E1)[0].rhs()
E1 = E1.subs(sustituciones)
show("De aquí sacamos E1 -> E1(r) = ", E1)

In [ ]:
show(deins[0,3].expr() == kappa*dTdd[0,3].expr())                                                           #Eq 48

In [ ]:
# Para la segunda derivada tenemos

show(ddeins[0,0].expr()," = ", kappa*ddTdd[0,0].expr())

In [ ]:
show(ddeins[1,1]," = ", kappa*ddTdd[1,1])

In [ ]:
show(ddeins[2,2]," = ", kappa*ddTdd[2,2])

In [ ]:
show(ddeins[3,3]," = ", kappa*ddTdd[3,3])

In [ ]:
show(ddeins[0,3]," = ", kappa*ddTdd[0,3])

In [ ]:
show(ddeins[1,2]," = ", kappa*ddTdd[1,2])

In [ ]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

In [ ]:
h

In [ ]:
ddeins[0,0]

In [ ]:
h0 = function('h0')(r,th)
k0 = function('k0')(r,th)
m0 = function('m0')(r,th)

h2 = function('h2')(r,th)
k2 = function('k2')(r,th)
m2 = function('m2')(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [ ]:
ddeins.apply_map(lambda cmp: cmp.subs(h==h0+h2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(k==k0+k2*(3*cos(th)^2-1)/2))
ddeins.apply_map(lambda cmp: cmp.subs(m==m0+m2*(3*cos(th)^2-1)/2))

In [ ]:
ddeins[0,0].subs(m==m0 + m2*(3*cos(th)^2-1)/2)